<a href="https://colab.research.google.com/github/breannashi/Data_Science_Bootcamp/blob/ml-tutorial/ML_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning Tutorial - Bioinformatics Bootcamp 2022


### 1.&nbsp;Prerequisite Knowledge and Table of Contents


This course focuses on a machine learning tutorial for people working in bioinformatics. We start with a clean synthetic dataset and apply statistical models in clustering, regression, and classification. This tutorial assumes a working knowledge of Python programming, linear algebra with NumPy, and Pandas for data analysis - especially the first two.


